# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# Helper functions

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data))) 

    return bs_replicates

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1)-np.mean(data_2)
    return diff

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

def permutation_sample(data_1, data_2):
    permuted_data = np.random.permutation(np.concatenate((data_1, data_2)))
    return permuted_data[:len(data_1)], permuted_data[len(data_1):]

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## Q1. What test is appropriate for this problem? Does CLT apply?

I would use 2-sample t-test. Two sample because we have 2 samples - black and white. T-test because the population standard deviation is not known.

Let's check if the CLT conditions are fulfilled by the sample.

1. Independent observations: Is sample size <=10% of the entire population?
<p>Sample size = 4870, 10% of 48700. United States labour market is definitely much larger than 48700.</p>

2. Large sample size: Is number of success and number of failure greater than 10 each?
<p> Calculated below, for both samples, number of success and failure is much larger than 10.
    So CLT is applicable.

In [6]:
print(np.sum(data[data.race=='b'].call))
print(np.sum(data[data.race=='w'].call))

157.0
235.0


## Q2. What are the null and alternate hypotheses?

<b><p>Null Hypothesis:</b> White and black races have same callback rate.</p>
<b>Alternate Hypothesis: </b> White and black races do not have same callback rate

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

In [8]:
# Your solution to Q3 here

## Q3. Compute margin of error, confidence interval, and p-value. 


In [9]:
w_call = w.call
b_call = b.call

In [10]:
# Percent callback
print('Call back for white-sounding',round(np.mean(w_call)*100,2),'percent')
print('Call back for black-sounding',round(np.mean(b_call)*100,2),'percent')

Call back for white-sounding 9.65 percent
Call back for black-sounding 6.45 percent


### Bootstrap approach

In [11]:
emp_diff = np.mean(w_call) - np.mean(b_call)

perm_reps = draw_perm_reps(w_call,b_call,diff_of_means,10000)
perm_p = np.sum(perm_reps>=emp_diff)/len(perm_reps)

ci = np.percentile(perm_reps,[2.5,97.5])
print('Confidence interval at 95% confidence level =',ci)
print('p-value = ',perm_p)

Confidence interval at 95% confidence level = [-0.01560576  0.01560576]
p-value =  0.0


### Frequentist Approach

In [12]:
w_std = np.std(w_call)
b_std = np.std(b_call)


# Confidence interval = mean +/- MOE at 95% CI

# z at 95% confidence
z=stats.norm.ppf(.975)

# Standard error of mean: sem
sem= np.sqrt(np.square(w_std)/len(w_call)+np.square(b_std)/len(b_call))

# Margin of error = z*SEM
moe = z*sem

print("Confidence Interval at 95% confidence level:",(emp_diff-moe),(emp_diff+moe))
print('Margin of error:',moe)

# Calculate p-value using 2-sample t test
p_value = stats.ttest_ind(b_call, w_call, equal_var=False)[1]
print('p-value is',p_value)

Confidence Interval at 95% confidence level: 0.01677785078904117 0.047287858921079985
Margin of error: 0.01525500406601941
p-value is 3.942941513645935e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## Q4. Write a story describing the statistical significance in the context or the original problem.

As per the research data, the white-sounding names received around 10% callback on their resume and the black-sounding names received around 6% callback. This difference in callback can be due to chance. To rule out chance from these results, both bootstrap as well as t-test was conducted. Both these test resulted in the conclusion that race does play a significant role in requests for interviews from employers.

## Q5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis here was focussed on identifying the role of race on callback success. As the role of other factors was not evaluated, we cannot say if race/name is most important factor.
In order to identify the most important factor, we need to analyse and compare the effect of other factors on callback too.